In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

pidf = pd.read_csv('/Users/marina/Desktop/project_2/data/clean/pidf.csv')
pidf.columns
pidf['variation'].unique()

array(['Test', 'other', 'Control'], dtype=object)

In [26]:
controldf = pidf[pidf['variation'] == 'Control']

# Create a dataframe where 'variation' is 'test'
testdf= pidf[pidf['variation'] == 'Test']

print("Original DataFrame:")
print(pidf.info())
print("\nControl DataFrame:")
print(controldf.info())
print("\nTest DataFrame:")
print(testdf.info())


Original DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158555 entries, 0 to 158554
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   client_id         158555 non-null  int64  
 1   clnt_tenure_yr    158555 non-null  int64  
 2   clnt_tenure_mnth  158555 non-null  int64  
 3   clnt_age          158555 non-null  int64  
 4   gendr             158555 non-null  object 
 5   num_accts         158555 non-null  int64  
 6   bal               158555 non-null  float64
 7   calls_6_mnth      158555 non-null  int64  
 8   logons_6_mnth     158555 non-null  int64  
 9   visitor_id        158555 non-null  object 
 10  visit_id          158555 non-null  object 
 11  start             145357 non-null  object 
 12  step_1            119705 non-null  object 
 13  step_2            104780 non-null  object 
 14  step_3            95529 non-null   object 
 15  confirm           90248 non-null   object 
 16  

In [31]:
# MIN MAX MEAN OF TIME ELAPSES BETWEEN STEPS FOR CONTROL GROUP

# Define the columns to be analyzed
time_diff_columns = ['start_step1', 'step1_step2', 'step2_step3', 'step3_confirm', 'start_confirm']

# Filter out rows with positive values in the specified columns
fildf = controldf[controldf[time_diff_columns] > 0]

# Calculate the min, max, and average for each time difference column
result = fildf[time_diff_columns].agg(['min', 'max', 'mean'])

# Display the result for each column
for column in time_diff_columns:
    print(f"Statistics for column: {column}")
    print("Min:", result.loc['min', column])
    print("Max:", result.loc['max', column])
    print("Mean:", result.loc['mean', column].round(2))
    print()

Statistics for column: start_step1
Min: 0.02
Max: 187.02
Mean: 0.96

Statistics for column: step1_step2
Min: 0.02
Max: 112.1
Mean: 0.9

Statistics for column: step2_step3
Min: 0.03
Max: 49.68
Mean: 1.67

Statistics for column: step3_confirm
Min: 0.02
Max: 157.68
Mean: 2.32

Statistics for column: start_confirm
Min: 0.55
Max: 189.18
Mean: 5.67



In [33]:
# MIN MAX MEAN OF TIME ELAPSES BETWEEN STEPS FOR TEST GROUP

# Define the columns to be analyzed
time_diff_columns = ['start_step1', 'step1_step2', 'step2_step3', 'step3_confirm', 'start_confirm']

# Filter out rows with positive values in the specified columns
fildf = testdf[testdf[time_diff_columns] > 0]

# Calculate the min, max, and average for each time difference column
result = fildf[time_diff_columns].agg(['min', 'max', 'mean'])

# Display the result for each column
for column in time_diff_columns:
    print(f"Statistics for column: {column}")
    print("Min:", result.loc['min', column])
    print("Max:", result.loc['max', column])
    print("Mean:", result.loc['mean', column].round(2))
    print()

Statistics for column: start_step1
Min: 0.02
Max: 102.05
Mean: 0.83

Statistics for column: step1_step2
Min: 0.02
Max: 113.93
Mean: 0.9

Statistics for column: step2_step3
Min: 0.02
Max: 187.2
Mean: 1.69

Statistics for column: step3_confirm
Min: 0.02
Max: 275.87
Mean: 2.37

Statistics for column: start_confirm
Min: 0.05
Max: 300.25
Mean: 5.45



In [40]:
# number of people in control that have completed all the steps for control

# Total number of rows
total_rows= controldf.shape[0]

# Number of rows without any null values in specified columns
non_null_rows = controldf.dropna(subset=['start', 'step_1', 'step_2', 'step_3', 'confirm']).shape[0]

# Number of rows with any null values in specified columns
null_rows = total_rows - non_null_rows

print("Total number of rows:", total_rows)
print("Number of rows without any null values in specified columns:", non_null_rows)
print("Number of rows with any null values in specified columns:", null_rows)

Total number of rows: 32181
Number of rows without any null values in specified columns: 14819
Number of rows with any null values in specified columns: 17362


In [42]:
# number of people in control that have completed all the steps for test

# Total number of rows
total_rows = testdf.shape[0]

# Number of rows without any null values in specified columns
non_null_rows= testdf.dropna(subset=['start', 'step_1', 'step_2', 'step_3', 'confirm']).shape[0]

# Number of rows with any null values in specified columns
null_rows = total_rows - non_null_rows

print("Total number of rows:", total_rows)
print("Number of rows without any null values in specified columns:", non_null_rows)
print("Number of rows with any null values in specified columns:", null_rows)

Total number of rows: 37122
Number of rows without any null values in specified columns: 17859
Number of rows with any null values in specified columns: 19263


In [46]:
# Total number of visit_id
total_visits = controldf['visit_id'].nunique()

# Function to calculate count and percentage
def calculate_non_null_and_null(controldf, non_null_col, null_cols):
    count = controldf[(controldf[non_null_col].notnull()) & (controldf[null_cols].isnull().all(axis=1))]['visit_id'].nunique()
    percentage = (count / total_visits) * 100
    return count, percentage

# Calculations
conresults = {}
conresults['start'] = calculate_non_null_and_null(controldf, 'start', ['step_1', 'step_2', 'step_3', 'confirm'])
conresults['step_1'] = calculate_non_null_and_null(controldf, 'step_1', ['start', 'step_2', 'step_3', 'confirm'])
conresults['step_2'] = calculate_non_null_and_null(controldf, 'step_2', ['start', 'step_1', 'step_3', 'confirm'])
conresults['step_3'] = calculate_non_null_and_null(controldf, 'step_3', ['start', 'step_1', 'step_2', 'confirm'])
conresults['confirm'] = calculate_non_null_and_null(controldf, 'confirm', ['start', 'step_1', 'step_2', 'step_3'])

# Print results
for key, value in conresults.items():
    print(f"For column {key}:")
    print(f"Count: {value[0]}, Percentage: {value[1]:.2f}%\n")

For column start:
Count: 7557, Percentage: 23.48%

For column step_1:
Count: 20, Percentage: 0.06%

For column step_2:
Count: 9, Percentage: 0.03%

For column step_3:
Count: 18, Percentage: 0.06%

For column confirm:
Count: 755, Percentage: 2.35%



In [66]:
#  for control group how manany people have reached to each step
total_visit_id = controldf['visit_id'].count()

# Create a dictionary to hold the results
results = {}

for column in columns_to_check:
    # Count the number of non-null values in the column
    non_null_count = controldf[column].notnull().sum()
    
    # Calculate the percentage of the total visit_id
    percentage = (non_null_count / total_visit_id) * 100
    
    # Store the results in the dictionary
    results[column] = {'count': non_null_count, 'percentage': percentage}

# Display the results
for column, result in results.items():
    print(f"{column}: {result['count']} non-null values ({result['percentage']:.2f}% of total visit_id)")


start: 30912 non-null values (96.06% of total visit_id)
step_1: 23558 non-null values (73.20% of total visit_id)
step_2: 20157 non-null values (62.64% of total visit_id)
step_3: 18321 non-null values (56.93% of total visit_id)
confirm: 16061 non-null values (49.91% of total visit_id)


In [68]:
#  for test group how many people have reached to each step
total_visit_id = testdf['visit_id'].count()

# Create a dictionary to hold the results
results = {}

for column in columns_to_check:
    # Count the number of non-null values in the column
    non_null_count = testdf[column].notnull().sum()
    
    # Calculate the percentage of the total visit_id
    percentage = (non_null_count / total_visit_id) * 100
    
    # Store the results in the dictionary
    results[column] = {'count': non_null_count, 'percentage': percentage}

# Display the results
for column, result in results.items():
    print(f"{column}: {result['count']} non-null values ({result['percentage']:.2f}% of total visit_id)")

start: 33153 non-null values (89.31% of total visit_id)
step_1: 28283 non-null values (76.19% of total visit_id)
step_2: 24507 non-null values (66.02% of total visit_id)
step_3: 22196 non-null values (59.79% of total visit_id)
confirm: 21737 non-null values (58.56% of total visit_id)


In [72]:
# test with all the steps percentage per gender
total_visit_id = testalldf['visit_id'].count()

# Group by the 'gender' column and count the number of visit_id for each gender
gender_counts = testalldf.groupby('gendr')['visit_id'].count()

# Calculate the percentage of each gender count
gender_percentages = (gender_counts / total_visit_id) * 100

# Combine the counts and percentages into a dataframe for better presentation
results = pd.DataFrame({
    'count': gender_counts,
    'percentage': gender_percentages
})
# Display the results
print("Number and percentage of visit_id values for each gender:")
print(results)

Number and percentage of visit_id values for each gender:
       count  percentage
gendr                   
F       5550   31.076768
M       6063   33.949269
U       6246   34.973963


In [70]:
# control with all the steps percentage per gender
total_visit_id = controlalldf['visit_id'].count()

# Group by the 'gender' column and count the number of visit_id for each gender
gender_counts = controlalldf.groupby('gendr')['visit_id'].count()

# Calculate the percentage of each gender count
gender_percentages = (gender_counts / total_visit_id) * 100

# Combine the counts and percentages into a dataframe for better presentation
results = pd.DataFrame({
    'count': gender_counts,
    'percentage': gender_percentages
})
# Display the results
print("Number and percentage of visit_id values for each gender:")
print(results)

Number and percentage of visit_id values for each gender:
       count  percentage
gendr                   
F       4576   30.879277
M       5078   34.266820
U       5165   34.853904


In [61]:
# Columns to check
columns_to_check = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Total number of visit_id
total_visit_id = controlalldf['visit_id'].count()

# Create a dictionary to hold the results
results = {}

for column in columns_to_check:
    # Count the number of positive values in the column
    positive_count = controlalldf[controlalldf[column] > 0][column].count()
    
    # Calculate the percentage of the total visit_id
    percentage = (positive_count / total_visit_id) * 100
    
    # Store the results in the dictionary
    results[column] = {'count': positive_count, 'percentage': percentage}

# Display the results
for column, result in results.items():
    print(f"{column}: {result['count']} positive values ({result['percentage']:.2f}% of total visit_id)")

TypeError: '>' not supported between instances of 'str' and 'int'

In [54]:
#control all df : control that have accomplished all steps

# Define the columns to check for non-null values
columns_to_check = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Create a new dataframe from df_control with non-null values in the specified columns
controlalldf = controldf.dropna(subset=columns_to_check)

# Display the dataframes
#print("Original Control DataFrame:")
#print(controldf)
#print("\nControl DataFrame with Non-Null Values in Specified Columns:")
#print(controlalldf)

In [56]:
#test all df : test that have accomplished all steps

# Define the columns to check for non-null values
columns_to_check = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Create a new dataframe from df_control with non-null values in the specified columns
testalldf = testdf.dropna(subset=columns_to_check)

# Display the dataframes
#print("Original Control DataFrame:")
#print(controldf)
#print("\nControl DataFrame with Non-Null Values in Specified Columns:")
#print(controlalldf)

In [ ]:
# error control, pivot control, que tiene solo valores negativos entre la diferencia de tiempo por cada paso

time_diff_columns = ['start_step1', 'step1_step2', 'step2_step3', 'step3_confirm', 'start_confirm']

# Filter out rows with negative values in the specified columns
errorpc_df = pc_df[(pc_df[time_diff_columns] < 0).any(axis=1)]

# Display the filtered DataFrame
display(errorpc_df)